#Testeo

In [2]:
#instalaciones requeridas
!pip install reverse_geocode
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install roboflow
!pip install streamlit

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-2uvos_de
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-2uvos_de
  Resolved https://github.com/facebookresearch/detectron2.git to commit 2a420edb307c9bdf640f036d3b196bed474b8593
  Preparing metadata (setup.py) ... done
  Using cached fvcore-0.1.5.post20221221-py3-none-any.whl
  Using cached iopath-0.1.9-py3-none-any.whl (27 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl (154 kB)
  Using cached black-24.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6174366 sha256=6de31b9e889bd84a63920fcf75ebfeb9d46daae5eff5b669b1f2915a60b8f58a
  Stored in directory: /tmp/pip-ephem-wheel-cache-16azy01n/wheels/47/e5/15/94c80df2ba85500c5d76599cc307c0a7079d0e221bb6fc4

In [3]:
#Librerías

#De uso general
import os #Para funcionalidades dependientes del sistema operativo
import cv2 #Para el procesamiento de imágenes
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt #Para crear gráficos
import numpy as np #Para operar funciones, vectores y matrices
import pandas as pd #Para manipulación y análisis de datos
from PIL import Image #Para abrir, manipular y guardar imágenes en varios formatos
import streamlit as st #Para la creación de aplicaciones web interactivas

#Conjunto de librerías para geolocalización
import reverse_geocode #Para convertir coordenadas geográficas (latitud y longitud) en direcciones o nombres de lugares
import folium #Para la creación de mapas interactivos
from IPython.display import display

%matplotlib inline
#Conjunto de librerías para la detección de objetos
import torch #ML basada en el cálculo automático de gradientes y en redes neuronales profundas.
import detectron2 #Detección de objetos.
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultTrainer
import logging #Para registrar eventos y mensajes durante la ejecución del programa
from detectron2.utils.logger import setup_logger
from roboflow import Roboflow #Ofrece datasets y herramientas para el entrenamiento de modelos


In [ ]:
# Definir el estilo CSS para el color de fondo
color_reto = "#FFD700"  # Amarillo
color_defi = "#ADD8E6"  # Azul claro
color_ries_con = "#90EE90"  # Verde claro

st.markdown("""
<style>
.centered {
    text-align: center;
}
</style>
<div class="centered">
    <h1>MonomerFinder: Identificación de microplásticos en imágenes</h1>
</div>
""", unsafe_allow_html=True)

# Menú desplegable para "¿Qué es MonomerFinder?"
with st.expander("¿Qué es MonomerFinder?"):
    st.markdown(f"<div style='background-color:{color_reto}; padding: 5px; border-radius: 5px;'>", unsafe_allow_html=True)
    st.write("MonomerFinder es una aplicación que facilita la identificación y clasificación de microplásticos presentes en imágenes capturadas por los propios usuarios. Para esto utiliza una IA que se encarga de analizar las imágenes y señalar la cantidad de microplásticos presentes y su tipo. Esta aplicación es el resultado del Reto 5 del Hackaton Coafina 2024: 'Microplásticos: un desafío ciudadano'.")
    st.markdown("</div>", unsafe_allow_html=True)

# Menú desplegable para "¿Qué son los microplásticos?"
with st.expander("¿Qué son los microplásticos?"):
    st.markdown(f"<div style='background-color:{color_defi}; padding: 5px; border-radius: 5px;'>", unsafe_allow_html=True)
    st.write("Los microplásticos son diminutas partículas de plástico, generalmente con tamaños menores a 5 mm, que se han convertido en un problema ambiental global. Provienen de diversas fuentes, como la degradación de objetos plásticos más grandes, productos cosméticos y fibras sintéticas de la ropa. Estos diminutos fragmentos contaminan océanos, ríos, suelos e incluso el aire, siendo ingeridos por organismos marinos y terrestres, lo que puede llevar a su acumulación en la cadena alimentaria. Su persistencia en el medio ambiente y sus potenciales efectos negativos en la salud humana y de los ecosistemas son motivo de creciente preocupación entre científicos y ambientalistas.")
    st.markdown("</div>", unsafe_allow_html=True)

# Menú desplegable adicional para "Fuentes de microplásticos"
with st.expander("Fuentes de microplásticos"):
    st.markdown(f"<div style='background-color:{color_ries_con}; padding: 5px; border-radius: 5px;'>", unsafe_allow_html=True)
    st.write("Los microplásticos pueden provenir de diversas fuentes, entre las que se destacan:")
    st.write("- Textiles:  El 70% de los textiles producidos hoy en día son de origen sintético y liberan grandes cantidades de microplásticos al medio ambiente. La liberación de fibras y sustancias sintéticas de la ropa se produce no solo a través del lavado y el uso, sino también durante la producción, el procesamiento y el transporte. Los geotextiles, que se utilizan comúnmente para sostener y reforzar las capas del suelo o como manto agrícola, también liberan microplásticos debido a la exposición a los rayos UV, daños físicos y un mantenimiento deficiente.")
    st.write("- Cosméticos y productos de cuidado personal: Un gran número de productos de cuidado personal todavía contienen microplásticos añadidos intencionadamente (por ejemplo, escarcha, microperlas en exfoliantes faciales y corporales). Estos productos ingresan al sistema de alcantarillado y a las plantas de tratamiento de aguas residuales. Aquí no se eliminan por completo y, por lo tanto, ingresan al medio ambiente.")
    st.write("- Pesca y acuicultura: Los microplásticos son liberados por: aguas grises de los buques que desembocan sin filtrar en el mar, la degradación de las líneas de pesca perdidas, pinturas y revestimientos marinos, residuos plásticos de un solo uso procedentes de la pesca y la acuicultura, y pérdida de contenedores marítimos con productos plásticos.")
    st.write("- Agricultura: Diversas fuentes agrícolas contribuyen a la propagación de microplásticos en el suelo, por ejemplo, a través de: el uso generalizado de películas de cultivo, tuberías de riego, comprimidos nutritivos, recubrimientos de semillas, lodos de depuradora de plantas de tratamiento de aguas residuales utilizados como fertilizante del suelo. Dado que la mitad de todos los lodos de depuradora de europa acaban de nuevo en tierra, se trata de un problema generalizado.")
    st.write("- Tráfico: La abrasión de los neumáticos, las marcas viales y los escombros de la carretera son fuentes de microplásticos ambientales. Se estima que solo la abrasión de los neumáticos genera más de 1,3 millones de toneladas de microplásticos en Europa cada año. El 'reciclaje' de neumáticos usados suele ser problemático, ya que los microplásticos se liberan directamente al medio ambiente a través de su uso en arrecifes artificiales, campos deportivos o patios escolares.")
    st.write("- Procesamiento de plásticos: Los pellets son perlas de plástico nuevas o recicladas que se utilizan como materia prima en la fabricación de la mayoría de los productos plásticos. Sin embargo, las pérdidas de gránulos, escamas y polvo de plástico se producen a lo largo de toda la cadena de valor de la producción, el transporte y el reciclaje de plástico. Además, el agua del proceso y las aguas residuales de la industria del plástico pueden estar muy contaminadas con microplásticos.")
    st.write("- Turismo: La eliminación inadecuada de los residuos de los turistas, los sistemas locales de gestión de residuos deficientes o inexistentes, la descarga incontrolada de aguas residuales de los cruceros y el consumo masivo de productos y envases de plástico de un solo uso en los países impulsados por el turismo, aumentan los aportes de microplásticos.")

    st.markdown("</div>", unsafe_allow_html=True)

# Software de MonomerFinder
st.write("""
<style>
.centered {
    text-align: center;
}
</style>
<div class="centered">
    <h1>¡Vamos a identificar y contar microplásticos!</h1>
</div>
""", unsafe_allow_html=True)
st.write("Recuerda:")
st.write("1-.Tener a la mano tu ubicación")
st.write("2-.La fotos deben ser de microplásticos filtrados es decir sin ningún otro material en lo posible")
st.write("3-.Las fotos deben ser claras para obtener mejores resultados")

# Solicitar al usuario que ingrese el nombre del archivo de imagen
st.markdown("""
    <style>
    .center {
        display: flex;
        justify-content: center;
    }
    </style>
    <div class="center">
        <h2>Sube tu imagen</h2>
    </div>
    """, unsafe_allow_html=True)
uploaded_file = st.file_uploader(" ", type=["jpg", "jpeg", "png"])

#Descarga y configuración del dataset
rf = Roboflow(api_key="Dlwgwe3psTHRPIuQlsbV")
project = rf.workspace("panats-mp-project").project("microplastic-dataset")
dataset = project.version(19).download("coco")

#Definición de rutas y nombres
DATA_SET_NAME = dataset.name.replace(" ", "-")
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"

#Registro del dataset
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME,
    metadata={},
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH,
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

#Validación del dataset
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=VALID_DATA_SET_NAME,
    metadata={},
    json_file=VALID_DATA_SET_ANN_FILE_PATH,
    image_root=VALID_DATA_SET_IMAGES_DIR_PATH
)

# Configuración del modelo
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
cfg.DATASETS.TEST = (VALID_DATA_SET_NAME,)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 3000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7  # Número de categorías de microplásticos


# Creación del entrenador
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Configuración del modelo
cfg.MODEL.WEIGHTS = "./output/model_final.pth"  # Ruta a los pesos del modelo entrenado
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # Umbral para la predicción
cfg.DATASETS.TEST = ("microplastic-dataset-valid", )

predictor = DefaultPredictor(cfg) #Predictor

# Ruta para guardar el archivo CSV
archivo_csv = 'microplastico_y_ubicaciones.csv'

# Leer la imagen
image_path = os.path.join(VALID_DATA_SET_IMAGES_DIR_PATH, uploaded_file)
image = cv2.imread(image_path)

if image is None:
    raise FileNotFoundError(f"La imagen '{uploaded_file}' no se encuentra en la ruta '{VALID_DATA_SET_IMAGES_DIR_PATH}'.")

# Detectar microplásticos
outputs = predictor(image)
instances = outputs["instances"].to("cpu")
pred_boxes = instances.pred_boxes.tensor.numpy()  # Coordenadas de las cajas delimitadoras

# Convertir la imagen a HSV
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Definir los rangos de color en HSV
color_ranges = {
    "Verde": ((0, 0, 200), (180, 30, 255)),
    "Azul": ((100, 150, 0), (140, 255, 255)),
    "Amarillo": ((20, 100, 100), (30, 255, 255)),
    "Blanco": ((40, 40, 40), (80, 255, 255)),
    "Rojo": ((0, 100, 100), (10, 255, 255)),
}

# Contar los microplásticos por color
color_counts = {color: 0 for color in color_ranges}

# Función para encontrar el color más cercano
def get_closest_color(hsv_image, x, y, w, h):
    roi = hsv_image[y:y+h, x:x+w]
    mean_color = cv2.mean(roi)[:3]
    min_dist = float('inf')
    closest_color = None
    for color, (lower, upper) in color_ranges.items():
        lower_bound = np.array(lower, dtype="uint8")
        upper_bound = np.array(upper, dtype="uint8")
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        mask_mean_color = cv2.mean(hsv_image, mask=mask)[:3]
        dist = np.linalg.norm(np.array(mean_color) - np.array(mask_mean_color))
        if dist < min_dist:
            min_dist = dist
            closest_color = color
    return closest_color

# Contar y asignar colores a cada microplástico detectado
microplastico_data = []
for box in pred_boxes:
    x1, y1, x2, y2 = box
    x, y, w, h = int(x1), int(y1), int(x2 - x1), int(y2 - y1)
    color = get_closest_color(hsv_image, x, y, w, h)
    if color:
        color_counts[color] += 1
        # Dibujar la caja delimitadora y el color en la imagen
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, color, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        # Agregar los datos al microplastico_data
        microplastico_data.append([color, x, y, w, h])

# Solicitar al usuario que ingrese el nombre del archivo de imagen
st.markdown("""
    <style>
    .center {
        display: flex;
        justify-content: center;
    }
    </style>
    <div class="center">
        <h2>Sube tu imagen</h2>
    </div>
    """, unsafe_allow_html=True)
uploaded_file = st.file_uploader(" ", type=["jpg", "jpeg", "png"])

# Ubicación en el mapa
st.markdown("""
    <style>
    .center {
        display: flex;
        justify-content: center;
    }
    </style>
    <div class="center">
        <h2>Ingresa tu ubicación</h2>
    </div>
    """, unsafe_allow_html=True)

def solicitar_datos():
  latitud = st.number_input('Latitud', value=0.0, format="%.6f")
  longitud = st.number_input('Longitud', value=0.0, format="%.6f")
  descripcion = st.text_input('Descripción de la ubicación (opcional)')

  # Ubicación en el mapa
  st.markdown("""
      <style>
      .center {
          display: flex;
          justify-content: center;
      }
      </style>
      <div class="center">
          <h2>Ubicación en el mapa</h2>
      </div>
      """, unsafe_allow_html=True)
  st.map(pd.DataFrame({'lat': [latitud], 'lon': [longitud], 'zoom': [1]}))

# Solicitar los datos
ubicacion_data = []
entrada = solicitar_datos()
if entrada:
    ubicacion_data.append(entrada)

# Contar los microplásticos por color y exportar a CSV
data = {
    "Color": [],
    "Conteo": []
}

total_microplasticos = 0
for color, count in color_counts.items():
    if count > 0:
        data["Color"].append(color)
        data["Conteo"].append(count)
        total_microplasticos += count

# Agregar el total de microplásticos
data["Color"].append("Total")
data["Conteo"].append(total_microplasticos)

# Convertir a DataFrame y exportar a CSV
df_microplastico = pd.DataFrame(data)

# Agregar los datos de microplásticos y ubicaciones en el mismo CSV
df_ubicaciones = pd.DataFrame(ubicacion_data, columns=["Latitud", "Longitud", "País", "Descripción"])
df_final = pd.concat([df_microplastico, df_ubicaciones], axis=1)

df_final.to_csv(archivo_csv, index=False)

st.write(f"Los datos se han guardado en '{archivo_csv}'.")

# Imprimir el resumen del conteo
st.write("Resumen del conteo de microplásticos:")
for color, count in color_counts.items():
    st.write(f"{color}: {count}")

# Imprimir el total de microplásticos
st.write(f"Total de microplásticos: {total_microplasticos}")

# Guardar la imagen con los rectángulos dibujados
output_image_path = os.path.join(VALID_DATA_SET_IMAGES_DIR_PATH, 'image_with_boxes.jpg')
cv2.imwrite(output_image_path, image)
st.write(f"Imagen con los rectángulos guardada en '{output_image_path}'.")

# Mostrar la imagen con las cajas delimitadoras y colores
cv2_imshow(image)  # Utiliza cv2.imshow en otros entornos
st.write("df_final")

st.write("¡Gracias! :)")

loading Roboflow workspace...
loading Roboflow project...
[07/21 00:55:38 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[07/21 00:55:39 d2.engine.train_loop]: Starting training from iteration 0


os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


[07/21 00:55:46 d2.utils.events]:  eta: 0:16:45  iter: 19  total_loss: 3.703  loss_cls: 1.971  loss_box_reg: 0.7059  loss_rpn_cls: 1.106  loss_rpn_loc: 0.09794    time: 0.3286  last_time: 0.3657  data_time: 0.0204  last_data_time: 0.0204   lr: 1.9981e-05  max_mem: 2009M
[07/21 00:55:52 d2.utils.events]:  eta: 0:16:34  iter: 39  total_loss: 2.89  loss_cls: 1.611  loss_box_reg: 0.8148  loss_rpn_cls: 0.4493  loss_rpn_loc: 0.119    time: 0.3307  last_time: 0.3700  data_time: 0.0142  last_data_time: 0.0057   lr: 3.9961e-05  max_mem: 2009M
[07/21 00:55:59 d2.utils.events]:  eta: 0:16:31  iter: 59  total_loss: 2.356  loss_cls: 1.131  loss_box_reg: 0.8071  loss_rpn_cls: 0.1267  loss_rpn_loc: 0.1383    time: 0.3328  last_time: 0.3671  data_time: 0.0099  last_data_time: 0.0333   lr: 5.9941e-05  max_mem: 2010M
[07/21 00:56:06 d2.utils.events]:  eta: 0:16:27  iter: 79  total_loss: 1.802  loss_cls: 0.6997  loss_box_reg: 0.8077  loss_rpn_cls: 0.1254  loss_rpn_loc: 0.07303    time: 0.3365  last_time: